Model Name: all-MiniLM-L6-v2

In [2]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer, util

In [3]:
data = pd.read_csv('../CSV/preped_data.csv')

labels = pd.read_csv('../CSV/labels.csv')

In [4]:
data.shape

(9494, 5)

In [5]:
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

In [6]:
#3 minutes to run
data['d_embendding'] = data['description'].apply(lambda x: model.encode(x)if isinstance(x, str) else None)
data['b_embendding'] = data['business_tags'].apply(lambda x: model.encode(x) if isinstance(x, str) else None)

In [7]:
labels['l_embendding'] = labels['label'].apply(lambda x: model.encode(x) if isinstance(x, str) else None)

In [8]:
description_similarity = []
business_tags_similarity = []

In [9]:
for i in range(data.shape[0]):
    if data['d_embendding'][i] is not None:
        description_similarity.append(cosine_similarity([data['d_embendding'][i]], labels['l_embendding'].tolist())[0])
    else:
        description_similarity.append([0] * len(labels))
    if data['b_embendding'][i] is not None:
        business_tags_similarity.append(cosine_similarity([data['b_embendding'][i]], labels['l_embendding'].tolist())[0])
    else:
        business_tags_similarity.append([0] * len(labels))

In [10]:
similarities = (np.array(description_similarity) + np.array(business_tags_similarity)) / 2

In [11]:
def get_top_labels(similarities, labels, top_n=3):
    top_indices = np.argsort(similarities, axis=1)[:, -top_n:][:, ::-1]
    top_labels = [[labels[i] for i in indices] for indices in top_indices]
    top_labels_with_scores = [[(labels[i], similarities[row_idx, i]) for i in indices] for row_idx, indices in enumerate(top_indices)]
    return top_labels_with_scores

In [12]:
similarities_array = np.vstack(similarities[:data.shape[0]])

In [13]:
data['top_labels'] = get_top_labels(similarities_array, labels['label'].tolist(), top_n=3)

In [14]:
data[['description', 'business_tags', 'top_labels']].to_csv('../CSV/labelled_data_sbert2.csv', index=False)